In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from skimage.io import imread

# Image Processing pipeline

We now know how to select and transfer data towards our device, let's apply some processing on it.

We will use a classic image processing pipeline: `blur > threshold > label > quantify`

### CPU - skimage pipeline

In [ ]:
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops_table

image = imread('./data/blobs.png').squeeze()

start = time.time()
blurred_image = gaussian(image, sigma=3)
binary_image = blurred_image > threshold_otsu(blurred_image)
labeled_image = label(binary_image)
props = regionprops_table(labeled_image, intensity_image=image, properties=
                          ['label', 'area', 'centroid', 'bbox', 'eccentricity', 'solidity', 'orientation', 'major_axis_length', 'minor_axis_length', 
                           'min_intensity', 'max_intensity', 'mean_intensity', 'std_intensity'])
print(f"Processing time: {time.time() - start} seconds")

fig, axs = plt.subplots(1, 4, figsize=(12, 6))
axs[0].imshow(image, cmap='gray')
axs[1].imshow(blurred_image, cmap='gray')
axs[2].imshow(binary_image, cmap='gray')
axs[3].imshow(labeled_image, cmap='nipy_spectral')
for ax in axs:
    ax.axis('off')
plt.show()

pd.DataFrame(props).head()

### GPU - clesperanto pipeline

Applying the same pipeline, this time using clesperanto. Naming are not exactly the same as we try to stick to wordy but clear operation name.

However, the API follow a specific pattern:

> output = cle.operation_name(input, output, args)

where:

- `cle` is the librairy handle
- `operation_name` is the name of the operation you want to apply
- `input` is the input data
- `output` is the output data
- `args` are the parameters of the operation (none, one, or more, generaly with default values)

If the `output` array is provided, the library will use it as it is. If left to `None`, the library will automatically create it on the device and return it.
In most usage, we will use the return `output` as it is easier but providing yourself the `output` variable allows you to have a more refine control on the device memory and output.

Each functions are documented, you can access though your IDE or using the `cle.operation_name?` idiom in a notebook, or use the [Documentation](https://clesperanto-doc.readthedocs.io)

#### Exercise 1: Reproduce the previous pipeline using clesperanto

Hint on available functions to do this: `gaussian_blur`, `threshold_otsu`, `connected_component_labeling`, `statistics_of_labelled_pixels`


In [ ]:
import pyclesperanto as cle
cle.select_device(1, device_type='gpu')

image = imread('./data/blobs.png').squeeze()

# TODO process the image with pyclesperanto

### GPU - cupy pipeline

Cupy is not an __image processing__ package, it comes with a large set of operation from the `ndimage` module from __scipy__ which allows us to perform several operation right on images but we still need to rely on some of the __scikit-image__ package.

> [NOTE]
> Not all operation can be performed on GPU, __cupy__ or __clesperanto__ are not replacement package for __scikit-image__ !

In [ ]:
try:
    import cupy as xp
except:
    import numpy as xp
    Warning("Cupy not found, using numpy instead.")

try:
    import cupyx.scipy.ndimage as xdi
except:
    import scipy.ndimage as xdi
    Warning("Cupy not found, using scipy instead.")

image = imread('./data/blobs.png').squeeze()

start = time.time()
gpu_img = xp.asarray(image)
blurred_image = xdi.gaussian_filter(gpu_img, sigma=3)
binary_image = blurred_image > threshold_otsu(blurred_image.get())
labeled_image, _ = xdi.label(binary_image)
props = regionprops_table(labeled_image.get(), intensity_image=gpu_img.get(), properties=
                          ['label', 'area', 'centroid', 'bbox', 'eccentricity', 'solidity', 'orientation', 'major_axis_length', 'minor_axis_length', 
                           'min_intensity', 'max_intensity', 'mean_intensity', 'std_intensity'])
print(f"Processing time: {time.time() - start} seconds")

fig, axs = plt.subplots(1, 4, figsize=(12, 6))
axs[0].imshow(gpu_img.get(), cmap='gray')
axs[1].imshow(blurred_image.get(), cmap='gray')
axs[2].imshow(binary_image.get(), cmap='gray')
axs[3].imshow(labeled_image.get(), cmap='nipy_spectral')
for ax in axs:
    ax.axis('off')
plt.show()

pd.DataFrame(props).head()